In [1]:
import pandas as pd
import torch
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from ndf import make_category_list
from ndf import TabularNumCatDataset
from torch.utils.data import DataLoader
from ndf import Numeric, TabularLayer
from ndf import NeuralDecisionForest, Forest
from ndf import ClassificationMetric
import torchmetrics
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from tqdm import tqdm
data_dir = Path("./../data")
df = pd.read_csv(data_dir.joinpath("./adult_train.csv"))
test = pd.read_csv(data_dir.joinpath("./adult_test.csv"))
test


num_cols = list(df.select_dtypes("number"))
target_col = ["income_bracket"]


num_cols = list(df.select_dtypes("number"))
target_col = ["income_bracket"]
cat_cols = sorted(list(set(list(df.columns)).difference(set(num_cols + target_col))))
total_cols = num_cols + target_col
data = df[total_cols]
y = df[target_col]
numX = df[num_cols]
catX = df[cat_cols]


testNumX = test[num_cols]
testCatX = test[cat_cols]
testY = test[target_col]


cat_cols = list(catX)
cat_list = make_category_list(catX, cat_cols)


num_info = Numeric(input_dim=numX.shape[1], output_dim=10, activation="relu", last_logit=True, layers=[25])
tabularLayer = TabularLayer(num_info, cat_list)


feature_layer = tabularLayer
feature_dim = tabularLayer.output_dim
forest_param = dict(
    n_tree=5, tree_depth=3, n_in_feature=feature_dim, tree_feature_rate=0.5, n_class=2, jointly_training=True
)
forest = Forest(**forest_param)
nnForest = NeuralDecisionForest(feature_layer=feature_layer, forest=forest)


def prepare_optim(model, lr):
    params = [p for p in model.parameters() if p.requires_grad]
    return torch.optim.Adam(params, lr=lr, weight_decay=1e-5)

optim = prepare_optim(nnForest, 1e-5)

education 0 16
gender 1 2
marital_status 2 7
native_country 3 42
occupation 4 15
race 5 5
relationship 6 6
workclass 7 9
0 16
1 2
2 7
3 42
4 15
5 5
6 6
7 9


In [2]:
import re 
save_dir = Path("./../model_v2")
weightdict = dict(torch.load(save_dir.joinpath("./model.pt")))
foodict = {k: v for k, v in weightdict.items() if re.search('category',k) is not None }

In [3]:
print(list(weightdict.keys()))

['feature_layer.num_emb_layer.0.weight', 'feature_layer.num_emb_layer.0.bias', 'feature_layer.num_emb_layer.2.weight', 'feature_layer.num_emb_layer.2.bias', 'feature_layer.cat_emb_layer.module_list.0.weight', 'feature_layer.cat_emb_layer.module_list.1.weight', 'feature_layer.cat_emb_layer.module_list.2.weight', 'feature_layer.cat_emb_layer.module_list.3.weight', 'feature_layer.cat_emb_layer.module_list.4.weight', 'feature_layer.cat_emb_layer.module_list.5.weight', 'feature_layer.cat_emb_layer.module_list.6.weight', 'feature_layer.cat_emb_layer.module_list.7.weight', 'feature_layer.numeric.0.weight', 'feature_layer.numeric.0.bias', 'feature_layer.numeric.2.weight', 'feature_layer.numeric.2.bias', 'feature_layer.category.module_list.0.weight', 'feature_layer.category.module_list.1.weight', 'feature_layer.category.module_list.2.weight', 'feature_layer.category.module_list.3.weight', 'feature_layer.category.module_list.4.weight', 'feature_layer.category.module_list.5.weight', 'feature_laye

In [4]:
for key , mat in foodict.items() :
    print(key, "/",mat.shape)

feature_layer.category.module_list.0.weight / torch.Size([16, 4])
feature_layer.category.module_list.1.weight / torch.Size([2, 2])
feature_layer.category.module_list.2.weight / torch.Size([7, 3])
feature_layer.category.module_list.3.weight / torch.Size([42, 7])
feature_layer.category.module_list.4.weight / torch.Size([15, 4])
feature_layer.category.module_list.5.weight / torch.Size([5, 3])
feature_layer.category.module_list.6.weight / torch.Size([6, 3])
feature_layer.category.module_list.7.weight / torch.Size([9, 3])


In [5]:
for k , v in dict(nnForest.feature_layer.cat_emb_layer.state_dict()).items() :
    print(k,"/",v.size())

module_list.0.weight / torch.Size([16, 4])
module_list.1.weight / torch.Size([2, 2])
module_list.2.weight / torch.Size([7, 3])
module_list.3.weight / torch.Size([42, 7])
module_list.4.weight / torch.Size([15, 4])
module_list.5.weight / torch.Size([5, 3])
module_list.6.weight / torch.Size([6, 3])
module_list.7.weight / torch.Size([9, 3])


In [6]:
#nnForest.feature_layer.cat_emb_layer.load_state_dict(torch.load(save_dir.joinpath("./model.pt")))

In [7]:
save_dir = Path("./../model_v2")
save_dir.mkdir(exist_ok=True)
nnForest.load(save_dir.joinpath("./model.pt"))
optim.load_state_dict(torch.load(save_dir.joinpath("./optim.pt")))